In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
%matplotlib notebook
from PIL import Image
from sklearn.utils import shuffle


!pip install nltk

#pre-processing
import re
import nltk
nltk.download('stopwords')
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer



#modeling
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier




#CV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from numpy import array
from sklearn.model_selection import KFold

!pip install pytorch_pretrained_bert
!pip install transformers
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import transformers as ppb # pytorch transformers
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 133kB 3.2MB/s 
     |████████████████████████████████| 573kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 11.0MB/s 
     |████████████████████████████████| 890kB 21.5MB/s 
     |████████████████████████████████| 3.7MB 28.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=57aeabe03e50ed89fd126a8de448f9af5b6b3b9052a8eaff6cd4b83dacd07fa5
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
link = 'https://drive.google.com/open?id=17cPPr1M0xHa8iv_tpBzQyNzkbd14iX98'

fluff, id = link.split('=')
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('791531.csv')  

17cPPr1M0xHa8iv_tpBzQyNzkbd14iX98


In [4]:
df = pd.read_csv('791531.csv', encoding='latin-1')
df.shape

(20050, 26)

In [5]:
df = df[['text', 'description', 'sidebar_color', 'gender:confidence', 'gender']]
#the model's main purpose is detecting gender, so we didn't pick 'location' feature
df.dropna(inplace = True)
df.drop(df[(df['gender'] == 'unknown') | 
           (df['gender'] == 'brand')].index, inplace = True)
df.drop(df[df['gender:confidence'] != 1].index, inplace = True)
df.shape

(8766, 5)

In [6]:
#two funcitons for normalization
def doc_features(doc):
    #HTML
    pattern = r"http\S+"
    documents = re.sub(pattern, " ", doc)
    
    #'-' letters
    documents = re.sub("[^a-zA-Z]"," ", doc)
    
    #To lower case
    words = documents.lower().split()
    
    #Stop words (nltk)
    stop = set(stopwords.words("english"))
    words = [w for w in words if not w in stop]
    
    #Stemmer & Return
    features = []
    stemmer = english_stemmer
    for word in words:
        features.append(stemmer.stem(word))
        
    return(features)
    
def text_cleaner(text):
    clean_text = []
    for i in text:
        clean_text.append(' '.join(doc_features(i)))
    
    return clean_text


df['text'] = text_cleaner(df['text']);
df['description'] = text_cleaner(df['description']);

#shuffle again
df = shuffle(df)
df.reset_index(drop=True, inplace= True)
m2_df_level0 = df[['text', 'description', 'gender']]

#set 75% train & 25% hideout

trainindex = int(len(m2_df_level0)*0.8)-1
hideoutindex = trainindex + 1

print(trainindex, hideoutindex)

m2_df_level0_train = m2_df_level0[['text','description','gender']].loc[:trainindex]
m2_df_level0_hideout = m2_df_level0[['text','description','gender']].loc[hideoutindex:]

print (m2_df_level0_train.shape, m2_df_level0_hideout.shape)

7011 7012
(7012, 3) (1754, 3)


In [0]:
def text_vec_convert2(X_train, X_test):
    #vectorizer = CountVectorizer(ngram_range = (1,1))
    #X_train = vectorizer.fit_transform(X_train)
    #X_test = vectorizer.transform(X_test)
    
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 
                                                    'distilbert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    
    tokenized_train = X_train.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    tokenized_test = X_test.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    
    
    padded_train = go_padded(tokenized_train)
    padded_test = go_padded(tokenized_test)
    
    features_train = transform(padded_train, model)
    features_test = transform(padded_test, model)

    return(features_train, features_test)
    
def go_padded(token):
    max_len = 0
    
    for i in token.values:
        if len(i) > max_len:
            max_len = len(i)
    
    padded = np.array([i + [0]*(max_len-len(i)) for i in token.values])
    return padded

def transform(padded, model):
    input_ids = torch.tensor(np.array(padded))
    with torch.no_grad():
        last_hidden_states = model(input_ids)  
    
    attention_mask = np.where(padded != 0, 1, 0)
    attention_mask.shape    
    input_ids = torch.tensor(padded)  
    attention_mask = torch.tensor(attention_mask)
    
    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)    
    features = last_hidden_states[0][:,0,:].numpy()    
    
    return features


def m2_lv0_modeling(X_train, y_train, X_column):
    
    X_sub_train, X_sub_test, y_sub_train, y_sub_test = train_test_split(X_train, y_train,
                                                                        test_size=0.50, random_state=100)
    estimators=[
        ('logistic',LogisticRegression(solver='liblinear',penalty='l2')),
        ('lasso',LogisticRegression(solver='liblinear',penalty='l1')),
        ('elasticnet',SGDClassifier(loss='log', penalty='elasticnet')),
        ('random_forest',RandomForestClassifier()),
        ('deep_nn',MLPClassifier()),
        ('knn',KNeighborsClassifier(n_neighbors=5,weights='distance',algorithm='auto')),
        #('MNB', MultinomialNB(alpha = 1))
        ]
    
    pipes=[]
    model_name = []
    pred_sub_df = pd.DataFrame({'clf_name':[], 'prob_male':[]})
    for model in estimators:
        pipe=Pipeline(steps=[(model)])
        pipe.fit(X_sub_train,y_sub_train)
        
        print(model[0], 'accuracy:', pipe.score(X_sub_test, y_sub_test))
        pred_sub = pipe.predict_proba(X_sub_test)
        pred_sub_df_temp = pd.DataFrame({'clf_name': X_column + '_' + model[0], 
                                         'prob_male':pred_sub[m, 1]} for m in range(len(pred_sub)))
        pred_sub_df = pred_sub_df.append(pred_sub_df_temp, sort= False)
        
        pipes.append(pipe)
        model_name.append(model[0])
     
    #return trained classfiers
    return pipes, model_name, pred_sub_df, y_sub_test


def m2_lv0_wrap(X_train, X_test, y_train):
    
    #take column name
    X_column = X_train.name
    
    #text vectorize
    X_train, X_test = text_vec_convert2(X_train, X_test)
    
    #modeling & get clf
    clfs, clf_names, pred_sub_df, y_sub_test = m2_lv0_modeling(X_train, y_train, X_column)
    
    #apply X_test to clf for prediction
    prediction_df = pd.DataFrame({'clf_name':[], 'prob_male':[]})
    for i , j in zip(clfs, clf_names):
        pred = i.predict_proba(X_test)
        prediction_df_temp = pd.DataFrame({'clf_name': X_column + '_' + j,
                                           'prob_male':pred[m, 1] } for m in range(len(pred)))
        prediction_df = prediction_df.append(prediction_df_temp, sort= False)
    #pivot prediction output
    prediction_df = prediction_df.pivot(index= None, columns= 'clf_name', values= 'prob_male')
    
    
    return prediction_df, pred_sub_df, y_sub_test
        
    

In [8]:
# y_test remains secret to clfs
m2_prediction_text, m2_subpred_text, y_sub_test = m2_lv0_wrap(m2_df_level0_train['text'], 
                                                                     m2_df_level0_hideout['text'],
                                                                     m2_df_level0_train['gender'])

m2_prediction_desc, m2_subpred_desc, y_sub_test = m2_lv0_wrap(m2_df_level0_train['description'], 
                                                                     m2_df_level0_hideout['description'],
                                                                     m2_df_level0_train['gender'])


logistic accuracy: 0.6026811180832858
lasso accuracy: 0.6100969766115231
elasticnet accuracy: 0.5855675984027382
random_forest accuracy: 0.6018254420992584


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


deep_nn accuracy: 0.5827153451226469
knn accuracy: 0.5593268682258985
logistic accuracy: 0.6631488876212208
lasso accuracy: 0.6602966343411295
elasticnet accuracy: 0.6329150028522533
random_forest accuracy: 0.6505989731888192


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


deep_nn accuracy: 0.6320593268682259
knn accuracy: 0.5892755276668569


In [9]:
# Gathering Train set from inner loop

m2_subpred_text = m2_subpred_text.pivot(index= None, columns= 'clf_name', values= 'prob_male')
m2_subpred_desc = m2_subpred_desc.pivot(index= None, columns= 'clf_name', values= 'prob_male')

y_sub_test.reset_index(drop= True, inplace= True)
m2_df_level1_train = pd.concat([m2_subpred_text, m2_subpred_desc, y_sub_test], axis= 1)
pd.set_option('display.max_colwidth', -1)
m2_df_level1_train.shape

# Gathering Valdate set from Hideout

m2_df_level0_hideout['gender'].reset_index(drop= True, inplace= True)
m2_df_level1_val = pd.concat([m2_prediction_text, m2_prediction_desc, m2_df_level0_hideout['gender']], axis= 1)
pd.set_option('display.max_colwidth', -1)
m2_df_level1_val.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


(1754, 13)

In [0]:
m2_X_train = m2_df_level1_train.loc[:, m2_df_level1_val.columns != 'gender']
m2_y_train = m2_df_level1_train['gender']

m2_X_val = m2_df_level1_val.loc[:, m2_df_level1_val.columns != 'gender'] 
m2_y_val = m2_df_level1_val['gender']

In [0]:
def m2_simple_lv1(X_train, X_test, y_train, y_test):

    estimators=[
        ('logistic',LogisticRegression(solver='liblinear',penalty='l2')),
        ('lasso',LogisticRegression(solver='liblinear',penalty='l1')),
        ('ridge',RidgeClassifier()),
        ('elasticnet',SGDClassifier(loss='log', penalty='elasticnet')),
        ('random_forest',RandomForestClassifier()),
        ('svc',LinearSVC()),
        ('deep_nn',MLPClassifier()),
        ('knn',KNeighborsClassifier(n_neighbors=3 ,weights='distance',algorithm='auto')),
        ('MNB', MultinomialNB(alpha = 1))
    ]
    
    scores=[]
    for model in estimators:
        pipe=Pipeline(steps=[(model)])
        pipe.fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        scores.append([score, model])
        
        
    return (scores)

In [12]:
m2_scores = m2_simple_lv1(m2_X_train, m2_X_val, m2_y_train, m2_y_val)
max(m2_scores)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[0.6915621436716077,
 ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
            intercept_scaling=1, loss='squared_hinge', max_iter=1000,
            multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
            verbose=0))]

In [0]:
#parameters = {'C': np.linspace(0.0001, 20, 30), 'penalty': ['l1', 'l2']}
#grid_search = GridSearchCV(LinearSVC(), parameters)
#grid_search.fit(m2_X_train, m2_y_train)

#print('best parameters: ', grid_search.best_params_)
#print('best scrores: ', grid_search.best_score_)


#c= 17.9 penalty l2

In [33]:
clf = LinearSVC(C= 17.9, penalty= 'l2')
clf.fit(m2_X_train, m2_y_train)
clf.score(m2_X_val, m2_y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6904218928164196

In [17]:
#@title
#try new data

link = 'https://drive.google.com/open?id=1x__GAttdEtRo8NjhItfaqATBlqxrcV7w'

fluff, id = link.split('=')
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('label_cleaned.csv')  

1x__GAttdEtRo8NjhItfaqATBlqxrcV7w


In [18]:
#@title
df2 = pd.read_csv('label_cleaned.csv')
df2.shape

(1336, 15)

In [0]:
#@title
df2 = df2[['text', 'description', 'profile_sidebar_fill_color', 'gender']]
df2.rename(columns={'profile_sidebar_fill_color':'sidebar_color'}, inplace=True)
#the model's main purpose is detecting gender, so we didn't pick 'location' feature
df2.dropna(inplace = True)
df2.drop(df2[(df2['gender'] == 'n') | 
           (df2['gender'] == 'b')].index, inplace = True)

df2['text'] = text_cleaner(df2['text']);
df2['description'] = text_cleaner(df2['description']);
df2 = shuffle(df2)
df2.reset_index(drop=True, inplace= True)

In [0]:
#@title
 df2_text_features, df2_desc_features = text_vec_convert2(df2['text'], df2['description'])
 